In [8]:
from azure.cognitiveservices.vision.customvision.training import training_api
from azure.cognitiveservices.vision.customvision.training.models import ImageUrlCreateEntry

# Replace with a valid key
training_key = "5c3d93078f114d54a9bfc635d5c91fed"
prediction_key = "bf415a8cfb3743d7b458b57fc67bba13"

trainer = training_api.TrainingApi(training_key)

# Create a new project
print ("Creating project...")
project = trainer.create_project("Table4Challenge1")

Creating project...


In [9]:
# Make two tags in the new project
hardshell_jacket_tag = trainer.create_tag(project.id, "hardshell jacket")
insulated_jacket_tag = trainer.create_tag(project.id, "insulated jacket")

In [11]:
base_image_url = "https://52.175.195.236:8000/user/hacker1/tree/gear_images/"
from pathlib import Path


print ("Adding images...")
# for image_num in range(1,10):
#     image_url = base_image_url + "hardshell_jackets/_{}.jpg".format(image_num)
#     trainer.create_images_from_urls(project.id, [ ImageUrlCreateEntry(url=image_url, tag_ids=[ hardshell_jackets_tag.id ] ) ])

# for image_num in range(1,10):
#     image_url = base_image_url + "insulated_jackets/_{}.jpg".format(image_num)
#     trainer.create_images_from_urls(project.id, [ ImageUrlCreateEntry(url=image_url, tag_ids=[ insulated_jackets_tag.id ] ) ])


# Alternatively, if the images were on disk in a folder called Images alongside the sample.py, then
# they can be added by using the following:

import os
hardshell_jackets_dir = "/home/hacker1/notebooks/gear_images/hardshell_jackets"
for image in os.listdir(os.fsencode(hardshell_jackets_dir)):
   with open(hardshell_jackets_dir + "/" + os.fsdecode(image), mode="rb") as img_data: 
       trainer.create_images_from_data(project.id, img_data, [ hardshell_jacket_tag.id ])

insulated_jackets_dir = "/home/hacker1/notebooks/gear_images/insulated_jackets"
for image in os.listdir(os.fsencode(insulated_jackets_dir)):
   with open(insulated_jackets_dir + "/" + os.fsdecode(image), mode="rb") as img_data: 
       trainer.create_images_from_data(project.id, img_data, [ insulated_jacket_tag.id ])
        
print("All done!")

Adding images...
All done!


In [12]:
import time

print ("Training...")
iteration = trainer.train_project(project.id)
while (iteration.status != "Completed"):
    iteration = trainer.get_iteration(project.id, iteration.id)
    print ("Training status: " + iteration.status)
    time.sleep(1)

# The iteration is now trained. Make it the default project endpoint
trainer.update_iteration(project.id, iteration.id, is_default=True)
print ("Done!")

Training...
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Completed
Done!


In [16]:
from azure.cognitiveservices.vision.customvision.prediction import prediction_endpoint
from azure.cognitiveservices.vision.customvision.prediction.prediction_endpoint import models

# Now there is a trained endpoint that can be used to make a prediction
print("start")
predictor = prediction_endpoint.PredictionEndpoint(prediction_key)
print(predictor)

test_img_url = "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSBOn_VN4eCj3JFgmzlDNm2oLmFCo6Agn9-eeHx330FLnsHG6nS"
results = predictor.predict_image_url(project.id, iteration.id, url=test_img_url)

# Alternatively, if the images were on disk in a folder called Images alongside the sample.py, then
# they can be added by using the following.
#
# Open the sample image and get back the prediction results.
# with open("Images\\test\\test_image.jpg", mode="rb") as test_data:
#     results = predictor.predict_image(project.id, test_data, iteration.id)

# Display the results.
for prediction in results.predictions:
    print ("\t" + prediction.tag_name + ": {0:.2f}%".format(prediction.probability * 100))

start
	insulated jacket: 100.00%
	hardshell jacket: 0.00%
